In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install vizdoom stable-baselines3[extra] imageio
!git clone https://github.com/mwydmuch/ViZDoom

In [ ]:
import shutil

# Path to original config
original_cfg = '/kaggle/working/ViZDoom/scenarios/deadly_corridor.cfg'

# Paths for new curriculum configs
s1_cfg = '/kaggle/working/ViZDoom/scenarios/deadly_corridor_s1.cfg'
s2_cfg = '/kaggle/working/ViZDoom/scenarios/deadly_corridor_s2.cfg'

# Copy the original file to new files
shutil.copyfile(original_cfg, s1_cfg)
shutil.copyfile(original_cfg, s2_cfg)

In [ ]:
cfg_path = '/kaggle/working/ViZDoom/scenarios/deadly_corridor.cfg'

# Read original lines
with open(cfg_path, 'r') as f:
    lines = f.readlines()

# Remove any existing available_game_variables lines (to avoid duplicates)
lines = [line for line in lines if not line.strip().startswith('available_game_variables')]

# Add the new line at the end (or wherever appropriate)
lines.append('available_game_variables = { HEALTH DAMAGE_TAKEN HITCOUNT SELECTED_WEAPON_AMMO }\n')

# Save to a new curriculum file, e.g., s1.cfg
new_cfg_path = '/kaggle/working/ViZDoom/scenarios/deadly_corridor_s1.cfg'
with open(new_cfg_path, 'w') as f:
    f.writelines(lines)

print(f"Updated config saved to: {new_cfg_path}")

In [ ]:
cfg_path = '/kaggle/working/ViZDoom/scenarios/deadly_corridor.cfg'

# Read original lines
with open(cfg_path, 'r') as f:
    lines = f.readlines()

# Remove any existing available_game_variables lines (to avoid duplicates)
lines = [line for line in lines if not line.strip().startswith('available_game_variables')]

# Replace doom_skill = 5 with doom_skill = 2
for i, line in enumerate(lines):
    if line.strip().startswith('doom_skill'):
        # Replace if it matches "doom_skill = 5"
        lines[i] = 'doom_skill = 0\n'  # Change skill level to 2

# Add the new available_game_variables line at the end
lines.append('available_game_variables = { HEALTH DAMAGE_TAKEN HITCOUNT SELECTED_WEAPON_AMMO }\n')

# Save to a new curriculum file for s2
new_cfg_path = '/kaggle/working/ViZDoom/scenarios/deadly_corridor_s2.cfg'
with open(new_cfg_path, 'w') as f:
    f.writelines(lines)

print(f"Updated config saved to: {new_cfg_path}")

In [ ]:
import numpy as np
from vizdoom import *
from gym import Env
from gym.spaces import Discrete, Box
import cv2
import random
import time

In [ ]:
!pip install shimmy>=2.0

In [ ]:
class VizDoomGym(Env):
    def __init__(self, render=False, config='/kaggle/working/ViZDoom/scenarios/deadly_corridor_s1.cfg'):
        super().__init__()
        self.game = DoomGame()
        self.game.load_config(config)
        self.game.set_window_visible(render)
        self.game.init()
        self.observation_space = Box(low=0, high=255, shape=(240,320,3), dtype=np.uint8)
        self.action_space = Discrete(7)
        self.damage_taken = 0
        self.hitcount = 0
        self.ammo = 52
    def step(self, action):
        actions = np.identity(7)
        movement_reward = self.game.make_action(actions[action], 4)
        reward = 0
        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.rgb_resize(state)
            game_variables = self.game.get_state().game_variables
            health, damage_taken, hitcount, ammo = game_variables
            damage_taken_delta = -damage_taken + self.damage_taken
            self.damage_taken = damage_taken
            hitcount_delta = hitcount - self.hitcount
            self.hitcount = hitcount
            ammo_delta = ammo - self.ammo
            self.ammo = ammo
            reward = movement_reward + damage_taken_delta*10 + hitcount_delta*200 + ammo_delta*5
            info = ammo
        else:
            state = np.zeros(self.observation_space.shape, dtype=np.uint8)
            info = 0
        info = {"info":info}
        done = self.game.is_episode_finished()
        return state, reward, done, False, info
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.rgb_resize(state), {}
    def rgb_resize(self, observation):
        obs = np.moveaxis(observation, 0, -1)
        resize = cv2.resize(obs, (320,240), interpolation=cv2.INTER_CUBIC)
        return resize
    def close(self):
        self.game.close()

In [ ]:
import os
from stable_baselines3.common.callbacks import BaseCallback
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, f'best_model_{self.n_calls}')
            self.model.save(model_path)
        return True

In [ ]:
CHECKPOINT_DIR = '/kaggle/working/train/train_corridor'
LOG_DIR = '/kaggle/working/logs/log_corridor'
callback = TrainAndLoggingCallback(check_freq=50000, save_path=CHECKPOINT_DIR)
env = VizDoomGym(render=False, config='/kaggle/working/ViZDoom/scenarios/deadly_corridor_s1.cfg')

In [ ]:
from stable_baselines3 import PPO
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.00001, n_steps=8192, clip_range=.1, gamma=.95, gae_lambda=.9)
model.learn(total_timesteps=500000, callback=callback)
model.save('/kaggle/working/vizdoom_ppo_model')

In [ ]:
import numpy as np
import imageio
from stable_baselines3 import PPO
from IPython.display import Video

# 1. Recreate your environment
env = VizDoomGym(render=False, config='/kaggle/working/ViZDoom/scenarios/deadly_corridor_s2.cfg')

# 2. Load your trained model (from checkpoint)
model = PPO.load('/kaggle/working/train/train_corridor/best_model_500000.zip', env=env)

frames = []
obs, _ = env.reset()
max_frames = 500
repeat = 5  # Slow playback

for _ in range(max_frames):
    state = env.game.get_state()
    if state is not None:
        frame = state.screen_buffer  # (C, H, W)
        frame = np.transpose(frame, (1, 2, 0))  # (H, W, C)
        frame = frame.astype(np.uint8)
        for _ in range(repeat):
            frames.append(frame)
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env.step(action)
    if done:
        obs, _ = env.reset()

# 3. Save video in the working directory (no /content/ on Kaggle!)
imageio.mimsave('vizdoom_gameplay.mp4', frames, fps=30, macro_block_size=None, quality=10)

# 4. Display the video (Kaggle supports this natively)
Video("vizdoom_gameplay.mp4", embed=True, width=512)

In [ ]:
!zip -r logs_corridor.zip /kaggle/working/logs/log_corridor

In [ ]:
!cp /kaggle/working/logs_corridor.zip /kaggle/working/

In [ ]:
!kill $(lsof -t -i:6006)

In [ ]:
%reload_ext tensorboard

In [ ]:
!ls /kaggle/working/logs/log_corridor

In [ ]:
%tensorboard --logdir /kaggle/working/logs/log_corridor --port 6006

In [ ]:
import numpy as np
import imageio

frames = []
obs, _ = env.reset()
max_frames = 500
repeat = 5  # Slow playback

for _ in range(max_frames):
    state = env.game.get_state()
    if state is not None:
        frame = state.screen_buffer  # (C, H, W)
        frame = np.transpose(frame, (1, 2, 0))  # (H, W, C)
        frame = frame.astype(np.uint8)
        for _ in range(repeat):
            frames.append(frame)
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env.step(action)
    if done:
        obs, _ = env.reset()

# Save video in the root directory (recommended for Kaggle)
imageio.mimsave('vizdoom_gameplay.mp4', frames, fps=30, macro_block_size=None, quality=10)

In [ ]:
from IPython.display import Video

# This will show the video player in the notebook cell
Video("vizdoom_gameplay.mp4", embed=True, width=512)